In [ ]:
!pip install opencv-python

In [ ]:
!pip uninstall easyocr
!pip install easyocr

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import easyocr

# Open the video file
video_path = 'UjiVideo.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error opening the video:", video_path)
    exit()

while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Resize the frame (you can adjust the dimensions as needed)
    frame = cv2.resize(frame, (812, 812))

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the frame
    img_blurred = cv2.GaussianBlur(frame, (5, 5), 1)

    # Apply Canny edge detection to the blurred frame
    edge_image_blurred = cv2.Canny(img_blurred, 60, 180)

    # Create a kernel for dilation
    kernel = np.ones((5, 5))

    # Dilate the edge-detected image
    imgDil = cv2.dilate(edge_image_blurred, kernel, iterations=2)

    # Find contours in the dilated image
    contours, _ = cv2.findContours(imgDil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw all contours on the original frame
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

    # Display the processed frame
    cv2_imshow(frame)

    # Use easyocr to extract text
    reader = easyocr.Reader(['en'], gpu=False)
    text_ = reader.readtext(frame)

    threshold = 0.25
    for t in text_:
      print(t)
      bbox, text, score = t

      if score > threshold:
        cv2.rectangle(frame, bbox[0], bbox[2], (0,255,0),5)
        cv2.putText(frame, text, bbox[0], cv2.FONT_HERSHEY_COMPLEX,0.65,(255,0,0),2)
    plt.imshow(frame)
    plt.show()

    # Check for user input to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()